In [2]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162671 sha256=c26ce7cd5cf772fa563247f75610ba93ae6c171ebd94f6f8215cbb5cc6cf8450
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [18]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import NMF

import time


In [5]:
data = Dataset.load_builtin('ml-100k')


In [8]:
model = SVD()

# Крос-валідація
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9373  0.9380  0.9350  0.9319  0.9398  0.9364  0.0027  
MAE (testset)     0.7394  0.7401  0.7369  0.7348  0.7417  0.7386  0.0024  
Fit time          1.59    1.45    1.48    1.44    1.49    1.49    0.05    
Test time         0.13    0.23    0.33    0.16    0.26    0.22    0.07    


{'test_rmse': array([0.93730019, 0.93803793, 0.9349554 , 0.93192662, 0.93981015]),
 'test_mae': array([0.73941989, 0.74010086, 0.73687768, 0.73484276, 0.74167287]),
 'fit_time': (1.5868258476257324,
  1.4484262466430664,
  1.475881576538086,
  1.4399785995483398,
  1.4944264888763428),
 'test_time': (0.13330388069152832,
  0.23247933387756348,
  0.32749128341674805,
  0.16088080406188965,
  0.2645690441131592)}

In [9]:
train_set, test_set = train_test_split(data, test_size=0.25)


In [15]:
# Запуск таймера для вимірювання часу виконання
start_time = time.time()

# Крос-валідація
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Визначення часу навчання та тестування
training_time = time.time() - start_time
testing_time = sum(results['test_time']) / len(results['test_time'])

# Виведення часу навчання та тестування
print(f"Training Time: {training_time} seconds")
print(f"Testing Time (avg): {testing_time} seconds")


# Виведення середніх значень RMSE та MAE на кожному з п'яти фолдів
for key in results.keys():
    print(f"{key}: {sum(results[key]) / len(results[key])}")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9314  0.9308  0.9373  0.9461  0.9404  0.9372  0.0057  
MAE (testset)     0.7332  0.7342  0.7395  0.7476  0.7395  0.7388  0.0051  
Fit time          1.90    1.49    1.53    1.49    1.51    1.58    0.16    
Test time         0.14    0.24    0.13    0.24    0.14    0.18    0.05    
Training Time: 9.840051889419556 seconds
Testing Time (avg): 0.17952523231506348 seconds
test_rmse: 0.9371818633974529
test_mae: 0.7388035353148174
fit_time: 1.5848032474517821
test_time: 0.17952523231506348


In [16]:
# Тренування моделі на тренувальному наборі
model.fit(train_set)

# Прогноз на тестовому наборі
predictions = model.test(test_set)

# Обчислення та виведення метрик точності
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.9352
MAE:  0.7370


0.7370011786360846

## Порівняння з іншими методами

In [19]:
# Визначення моделі NMF
nmf_model = NMF(n_factors=50, random_state=0)

# Оцінка за допомогою крос-валідації
start_time = time.time()
results_nmf = cross_validate(nmf_model, data, cv=5, verbose=True)
training_time_nmf = time.time() - start_time
testing_time_nmf = sum(results_nmf['test_time']) / len(results_nmf['test_time'])

# Оцінка точності
rmse_nmf = results_nmf['test_rmse'].mean()
mae_nmf = results_nmf['test_mae'].mean()

# Виведення результатів
print(f"NMF - RMSE: {rmse_nmf}, MAE: {mae_nmf}")
print(f"Training Time (NMF): {training_time_nmf} seconds")
print(f"Testing Time (avg, NMF): {testing_time_nmf} seconds")


Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0324  1.0313  1.0242  1.0300  1.0293  1.0294  0.0028  
MAE (testset)     0.7852  0.7852  0.7804  0.7873  0.7853  0.7847  0.0023  
Fit time          4.12    3.58    3.28    3.54    4.13    3.73    0.34    
Test time         0.20    0.24    0.11    0.31    0.10    0.19    0.08    
NMF - RMSE: 1.029432581074921, MAE: 0.7846768348900545
Training Time (NMF): 20.704153537750244 seconds
Testing Time (avg, NMF): 0.19218125343322753 seconds


# Отримані результати показують наступне:

##SVD:

RMSE: 0.9370 (середньоквадратична помилка)
MAE: 0.7383 (середня абсолютна помилка)
Час навчання: 1.64 секунд
Час тестування (середній): 0.25 секунд

##NMF:

RMSE: 1.0294
MAE: 0.7847
Час навчання: 3.73 секунди
Час тестування (середній): 0.19 секунд

#Висновок:
Обидва методи мають схожі значення RMSE та MAE, і обидва можуть бути використані для побудови рекомендаційних систем. Вибір між ними може залежати від таких чинників:

1) Час виконання:

Якщо важливо мінімізувати час виконання, SVD може бути кращим варіантом, оскільки він навчається швидше.

2) Точність:

Якщо точність є важливішою, то можливо варто розглянути SVD, оскільки він має трохи менші значення RMSE та MAE.

3) Загальна продуктивність:

В залежності від розміру набору даних та конкретних вимог, можна експериментувати з рызними методами та порівнювати їх продуктивність.
Обидва методи мають свої переваги та недоліки, і вибір може залежити від конкретної ситуації. У даному випадку, з огляду на час виконання та аналіз точності, метод SVD може бути більш оптимальним. Однак при роботі з більшими обсягами даних, або при інших конкретних вимогах, це може змінитися.